# DATA INTEGRATION
A notebook to integrate multiple data sources

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import copy
import pandas as pd
import numpy as np
import os
PRJ_ROOT_DIR = os.path.dirname(os.path.abspath(''))

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import seaborn as sns

In [2]:
# Where to save the figures
NOTE_ROOT_DIR = os.path.abspath('')
CHAPTER_ID = "03_data_integrating"
IMAGES_PATH = os.path.join(NOTE_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## **1 - Load the Data**

### 1.1 - EPL Match

In [3]:
# Load EPL match data
df_match = pd.read_csv(os.path.join(PRJ_ROOT_DIR, "data", "tabular", "exploratory", "matches.csv"))
df_match = df_match[df_match["season"] != "2022/23"] # Exclude season 2022/23
df_match.head()

,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,home/mf_2,home/mf_3,...,away/mf_2,away/mf_3,away/mf_4,away/fw_0,away/fw_1,away/fw_2,home/name,away/name,home_result,season
0,37096.0,55459.0,166640.0,219924.0,81012.0,NaN,101537.0,57531.0,86934.0,54102.0,...,20664.0,NaN,NaN,103955.0,205651.0,103025.0,West Ham United,Manchester City,lose,2019/20
1,37915.0,38290.0,55605.0,173904.0,158534.0,NaN,157668.0,62974.0,45268.0,231372.0,...,85242.0,148508.0,193488.0,213345.0,NaN,NaN,Tottenham Hotspur,Aston Villa,win,2019/20
2,116535.0,171287.0,97032.0,122798.0,169187.0,NaN,116643.0,41733.0,56979.0,NaN,...,71738.0,90440.0,78607.0,57127.0,NaN,NaN,Liverpool,Norwich City,win,2019/20
3,98747.0,68983.0,17761.0,51927.0,39487.0,NaN,40145.0,60586.0,433154.0,60551.0,...,101178.0,213482.0,NaN,84939.0,200439.0,83283.0,Burnley,Southampton,win,2019/20
4,51940.0,184667.0,95658.0,106760.0,214590.0,NaN,74208.0,176297.0,109322.0,156689.0,...,88894.0,49579.0,184341.0,173879.0,NaN,NaN,Manchester United,Chelsea,win,2019/20


In [4]:
# List positions
all_pos = list(df_match.columns)[:-4]
all_pos

['home/gk_0',
 'home/df_0',
 'home/df_1',
 'home/df_2',
 'home/df_3',
 'home/df_4',
 'home/mf_0',
 'home/mf_1',
 'home/mf_2',
 'home/mf_3',
 'home/mf_4',
 'home/fw_0',
 'home/fw_1',
 'home/fw_2',
 'away/gk_0',
 'away/df_0',
 'away/df_1',
 'away/df_2',
 'away/df_3',
 'away/mf_0',
 'away/mf_1',
 'away/mf_2',
 'away/mf_3',
 'away/mf_4',
 'away/fw_0',
 'away/fw_1',
 'away/fw_2']

In [5]:
# List position types
position = ['home/gk', 'home/df', 'home/mf', 'home/fw',
            'away/gk', 'away/df', 'away/mf', 'away/fw']

In [6]:
# Fill empty positions with ghost id (-1)
df_match = df_match.fillna(-1)
df_match[df_match["home/df_4"] == -1].head()

,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,home/mf_2,home/mf_3,...,away/mf_2,away/mf_3,away/mf_4,away/fw_0,away/fw_1,away/fw_2,home/name,away/name,home_result,season
0,37096.0,55459.0,166640.0,219924.0,81012.0,-1.0,101537.0,57531.0,86934.0,54102.0,...,20664.0,-1.0,-1.0,103955.0,205651.0,103025.0,West Ham United,Manchester City,lose,2019/20
1,37915.0,38290.0,55605.0,173904.0,158534.0,-1.0,157668.0,62974.0,45268.0,231372.0,...,85242.0,148508.0,193488.0,213345.0,-1.0,-1.0,Tottenham Hotspur,Aston Villa,win,2019/20
2,116535.0,171287.0,97032.0,122798.0,169187.0,-1.0,116643.0,41733.0,56979.0,-1.0,...,71738.0,90440.0,78607.0,57127.0,-1.0,-1.0,Liverpool,Norwich City,win,2019/20
3,98747.0,68983.0,17761.0,51927.0,39487.0,-1.0,40145.0,60586.0,433154.0,60551.0,...,101178.0,213482.0,-1.0,84939.0,200439.0,83283.0,Burnley,Southampton,win,2019/20
4,51940.0,184667.0,95658.0,106760.0,214590.0,-1.0,74208.0,176297.0,109322.0,156689.0,...,88894.0,49579.0,184341.0,173879.0,-1.0,-1.0,Manchester United,Chelsea,win,2019/20


### 1.2 - EPL Player

In [15]:
# Load EPL player data (i.e. of PCA/LDA and so on)
df_player = pd.read_csv(os.path.join(PRJ_ROOT_DIR, "data", "tabular", "clean", 
                                     "players.csv")) # <-- change to other embeddings if needed
df_player.head()

,defence/clean_sheets,defence/goals_conceded,defence/tackles,defence/tackle_success_%,defence/last_man_tackles,defence/blocked_shots,defence/interceptions,defence/clearances,defence/headed_clearance,defence/clearances_off_line,...,goalkeeping/goal_kicks,defence/recoveries,defence/duels_won,defence/duels_lost,teamplay/passes,teamplay/crosses,teamplay/accurate_long_balls,id,name,season
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,468236.0,Yan Couto,2021/22
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,447880.0,Josh Wilson-Esbrand,2021/22
2,0.0,0.0,12.0,50.0,0.0,6.0,8.0,5.0,4.0,0.0,...,0.0,58.0,73.0,71.0,273.0,11.0,5.0,216646.0,Yoane Wissa,2021/22
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,491556.0,Charlie Whitaker,2021/22
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,515503.0,Jack Wells-Morrison,2021/22


In [16]:
# Add average performance for each skill group (OPTIONAL, for original data)
skill_grps = ["defence", "attack", "teamplay", "goalkeeping", "discipline"]
df_player_temp = df_player.iloc[:, :-3]
for grp in skill_grps:
    df_player_temp[grp] = df_player_temp[[col for col in df_player if col.startswith(grp)]].mean(axis=1)
df_player_temp[["id", "name", "season"]] = df_player[["id", "name", "season"]]
df_player = df_player_temp
df_player.head()

,defence/clean_sheets,defence/goals_conceded,defence/tackles,defence/tackle_success_%,defence/last_man_tackles,defence/blocked_shots,defence/interceptions,defence/clearances,defence/headed_clearance,defence/clearances_off_line,...,teamplay/crosses,teamplay/accurate_long_balls,defence,attack,teamplay,goalkeeping,discipline,id,name,season
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,468236.0,Yan Couto,2021/22
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,447880.0,Josh Wilson-Esbrand,2021/22
2,0.0,0.0,12.0,50.0,0.0,6.0,8.0,5.0,4.0,0.0,...,11.0,5.0,19.833333,8.519167,34.455556,0.0,3.75,216646.0,Yoane Wissa,2021/22
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,491556.0,Charlie Whitaker,2021/22
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,515503.0,Jack Wells-Morrison,2021/22


## 2 - Integrate the Data

In [17]:
# Clone match data for safe integration
df_integrate = df_match.copy()
df_integrate.head()

,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,home/mf_2,home/mf_3,...,away/mf_2,away/mf_3,away/mf_4,away/fw_0,away/fw_1,away/fw_2,home/name,away/name,home_result,season
0,37096.0,55459.0,166640.0,219924.0,81012.0,-1.0,101537.0,57531.0,86934.0,54102.0,...,20664.0,-1.0,-1.0,103955.0,205651.0,103025.0,West Ham United,Manchester City,lose,2019/20
1,37915.0,38290.0,55605.0,173904.0,158534.0,-1.0,157668.0,62974.0,45268.0,231372.0,...,85242.0,148508.0,193488.0,213345.0,-1.0,-1.0,Tottenham Hotspur,Aston Villa,win,2019/20
2,116535.0,171287.0,97032.0,122798.0,169187.0,-1.0,116643.0,41733.0,56979.0,-1.0,...,71738.0,90440.0,78607.0,57127.0,-1.0,-1.0,Liverpool,Norwich City,win,2019/20
3,98747.0,68983.0,17761.0,51927.0,39487.0,-1.0,40145.0,60586.0,433154.0,60551.0,...,101178.0,213482.0,-1.0,84939.0,200439.0,83283.0,Burnley,Southampton,win,2019/20
4,51940.0,184667.0,95658.0,106760.0,214590.0,-1.0,74208.0,176297.0,109322.0,156689.0,...,88894.0,49579.0,184341.0,173879.0,-1.0,-1.0,Manchester United,Chelsea,win,2019/20


In [18]:
# Integrate critical performance index to corresponding player slots in match data
players_to_performance = {"gk": "goalkeeping", "df": "defence", "mf": "teamplay", "fw": "attack"} # critical attributes
score_players_df = pd.DataFrame()
for pos in position:  #home/gk
    score_df = df_integrate.copy()
    score_df = score_df[[col for col in score_df.columns if pos in col]+["season"]]
    score_col_idx = [col_id for (posit, col_id) in players_to_performance.items() if posit in pos]
    position_players_score = df_player[['id', 'season'] + score_col_idx]
    count=0
    for i in range(len(all_pos)):
      if pos in all_pos[i]:
        count += 1
        score_df = score_df.merge(position_players_score, how='left', left_on=[all_pos[i], 'season'], right_on=['id', 'season'], suffixes=(f"_{i-1}", f"_{i}"))                                   
    multiplier = 1
    #if "df" in pos or "mf" in pos:
    #  multiplier = -1
    score_df[score_df.columns[:count]] = multiplier * score_df[score_df.columns[count+2::2]]
    score_df.drop(columns=score_df.columns[count+1:], inplace=True)
    score_players_df = pd.concat([score_players_df, score_df], axis=1)

score_players_df.drop(columns='season', inplace=True)
score_players_df = pd.concat([score_players_df, df_integrate[["home/name", "away/name", "season", "home_result"]]], axis=1)
columns_titles = ["home/name", "away/name",
                  'home/gk_0', 'home/df_0', 'home/df_1', 'home/df_2', 'home/df_3',
                  'home/df_4', 'home/mf_0', 'home/mf_1', 'home/mf_2', 'home/mf_3',
                  'home/mf_4', 'home/fw_0', 'home/fw_1', 'home/fw_2', 'away/gk_0',
                  'away/df_0', 'away/df_1', 'away/df_2', 'away/df_3', 'away/mf_0',
                  'away/mf_1', 'away/mf_2', 'away/mf_3', 'away/mf_4', 'away/fw_0',
                  'away/fw_1', 'away/fw_2',
                  "home_result", "season"]
score_players_df = score_players_df.reindex(columns=columns_titles)
score_players_df = score_players_df.fillna(-1e10)
print(score_players_df.columns)
score_players_df.head()

Index(['home/name', 'away/name', 'home/gk_0', 'home/df_0', 'home/df_1',
       'home/df_2', 'home/df_3', 'home/df_4', 'home/mf_0', 'home/mf_1',
       'home/mf_2', 'home/mf_3', 'home/mf_4', 'home/fw_0', 'home/fw_1',
       'home/fw_2', 'away/gk_0', 'away/df_0', 'away/df_1', 'away/df_2',
       'away/df_3', 'away/mf_0', 'away/mf_1', 'away/mf_2', 'away/mf_3',
       'away/mf_4', 'away/fw_0', 'away/fw_1', 'away/fw_2', 'home_result',
       'season'],
      dtype='object')


,home/name,away/name,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,...,away/mf_0,away/mf_1,away/mf_2,away/mf_3,away/mf_4,away/fw_0,away/fw_1,away/fw_2,home_result,season
0,West Ham United,Manchester City,50.250,38.277778,30.555556,50.777778,43.222222,-1.000000e+10,116.200000,58.064444,...,318.187778,277.387778,182.547778,-1.000000e+10,-1.000000e+10,19.884167,2.103417e+01,1.502750e+01,lose,2019/20
1,Tottenham Hotspur,Aston Villa,44.250,30.777778,54.055556,45.944444,22.388889,-1.000000e+10,209.072222,71.737778,...,102.060000,185.120000,89.518889,5.196778e+01,6.756556e+01,10.020000,-1.000000e+10,-1.000000e+10,win,2019/20
2,Liverpool,Norwich City,50.750,40.055556,9.111111,45.333333,47.055556,-1.000000e+10,197.262222,208.248889,...,128.642222,195.342222,47.536667,7.560444e+01,2.106011e+02,16.275833,-1.000000e+10,-1.000000e+10,win,2019/20
3,Burnley,Southampton,71.875,27.111111,83.000000,64.666667,33.833333,-1.000000e+10,114.307778,354.360000,...,163.267778,135.430000,232.426667,1.277778e+01,-1.000000e+10,19.215000,7.594167e+00,1.076083e+01,win,2019/20
4,Manchester United,Chelsea,68.625,50.611111,74.277778,30.722222,58.500000,-1.000000e+10,136.847778,113.372222,...,278.423333,228.695556,80.825556,4.062667e+01,1.640600e+02,19.036667,-1.000000e+10,-1.000000e+10,win,2019/20


In [19]:
# Transpose the merged critical performane data
df_one_perf = score_players_df.T.copy()
df_one_perf.head()

,0,1,2,3,4,5,6,7,8,9,...,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075
home/name,West Ham United,Tottenham Hotspur,Liverpool,Burnley,Manchester United,Leicester City,Bournemouth,Watford,Crystal Palace,Newcastle United,...,Burnley,Southampton,Arsenal,Crystal Palace,Wolverhampton Wanderers,West Ham United,Southampton,Manchester United,Liverpool,Newcastle United
away/name,Manchester City,Aston Villa,Norwich City,Southampton,Chelsea,Wolverhampton Wanderers,Sheffield United,Brighton and Hove Albion,Everton,Arsenal,...,Watford,Manchester City,Burnley,Liverpool,Southampton,Norwich City,Brentford,Wolverhampton Wanderers,Leeds United,Manchester City
home/gk_0,50.25,44.25,50.75,71.875,68.625,67.5,82.375,74.875,77.125,82.0,...,80.125,40.125,247.375,65.125,86.5,75.875,40.125,74.125,66.625,53.75
home/df_0,38.277778,30.777778,40.055556,27.111111,50.611111,45.444444,52.277778,48.666667,44.555556,25.0,...,70.944444,51.166667,25.111111,47.222222,39.444444,35.444444,22.222222,8.111111,48.777778,20.055556
home/df_1,30.555556,54.055556,9.111111,83.0,74.277778,152.611111,48.555556,45.555556,36.666667,38.611111,...,38.0,27.277778,47.833333,53.111111,47.388889,37.555556,70.222222,30.777778,34.555556,38.944444


In [20]:
# Complete critical attributes to match data (for merge ordering)
df_match_tranposed = df_match.T.copy() # [index: home/fw_0, column: 0, 1, 2]
for col in df_one_perf.columns:   # 0, 1
    for pos in position:  # home/gk, home/fw
        pos_order = [row for row in df_match_tranposed.index if pos in row]
        sorted_pos = df_match_tranposed[col].loc[pos_order].sort_values(ascending=False, ignore_index=False,
                                  key=lambda x: df_one_perf[col].loc[[row for row in df_one_perf.index if pos in row]].astype(float))
        re_cols = dict(zip(sorted_pos.index, pos_order))
        df_match_tranposed[col].loc[pos_order] = sorted_pos.rename(index=re_cols)
df_match_tranposed.reindex()
df_match_tranposed.head()

,0,1,2,3,4,5,6,7,8,9,...,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075
home/gk_0,37096.0,37915.0,116535.0,98747.0,51940.0,17745.0,225321.0,9089.0,40836.0,67089.0,...,98747.0,40383.0,225321.0,40836.0,149065.0,37096.0,40383.0,51940.0,116535.0,67089.0
home/df_0,219924.0,55605.0,169187.0,17761.0,95658.0,218031.0,56917.0,60232.0,55494.0,119471.0,...,17761.0,450527.0,198869.0,244723.0,214048.0,60232.0,450527.0,214590.0,60914.0,101148.0
home/df_1,81012.0,173904.0,122798.0,51927.0,214590.0,37642.0,126184.0,41338.0,74230.0,101148.0,...,51927.0,171771.0,226597.0,209036.0,94147.0,164555.0,171771.0,90152.0,97032.0,114243.0
home/df_2,55459.0,38290.0,171287.0,39487.0,184667.0,111931.0,223911.0,54484.0,58786.0,106618.0,...,192290.0,158534.0,192895.0,55494.0,510362.0,55459.0,212721.0,106760.0,169187.0,58845.0
home/df_3,166640.0,158534.0,97032.0,68983.0,106760.0,172850.0,-1.0,40868.0,19188.0,-1.0,...,39487.0,244560.0,156074.0,174874.0,-1.0,219924.0,-1.0,76359.0,122798.0,56983.0


In [21]:
# Re-transpose the order merge data
ordered_epl = df_match_tranposed.T.copy()
ordered_epl.head()

,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,home/mf_2,home/mf_3,...,away/mf_2,away/mf_3,away/mf_4,away/fw_0,away/fw_1,away/fw_2,home/name,away/name,home_result,season
0,37096.0,219924.0,81012.0,55459.0,166640.0,-1.0,204480.0,101537.0,86934.0,57531.0,...,20664.0,-1.0,-1.0,205651.0,103955.0,103025.0,West Ham United,Manchester City,lose,2019/20
1,37915.0,55605.0,173904.0,38290.0,158534.0,-1.0,157668.0,45268.0,231372.0,62974.0,...,85242.0,193488.0,148508.0,213345.0,-1.0,-1.0,Tottenham Hotspur,Aston Villa,win,2019/20
2,116535.0,169187.0,122798.0,171287.0,97032.0,-1.0,56979.0,41733.0,116643.0,-1.0,...,193111.0,90440.0,71738.0,57127.0,-1.0,-1.0,Liverpool,Norwich City,win,2019/20
3,98747.0,17761.0,51927.0,39487.0,68983.0,-1.0,60586.0,60551.0,433154.0,40145.0,...,78056.0,213482.0,-1.0,84939.0,83283.0,200439.0,Burnley,Southampton,win,2019/20
4,51940.0,95658.0,214590.0,184667.0,106760.0,-1.0,74208.0,195851.0,176297.0,156689.0,...,184341.0,88894.0,49579.0,173879.0,-1.0,-1.0,Manchester United,Chelsea,win,2019/20


In [22]:
# Add empty slots as players 
all_pos_home = ['home/name', 'away/name', 'home/gk_0', 'home/df_0', 'home/df_1',
                'home/df_2', 'home/df_3', 'home/df_4', 'home/mf_0', 'home/mf_1',
                'home/mf_2', 'home/mf_3', 'home/mf_4', 'home/fw_0', 'home/fw_1',
                'home/fw_2']

all_season = df_match['season'].unique().tolist() 
wno_players = df_player.copy()
wno_players.drop(columns='name', inplace=True)
for season in all_season:
    null_record = {'id': -1, 'season': season}
    for feat in wno_players.columns[:-2]:
        null_record[feat] = -100.
    wno_players = wno_players.append(null_record, ignore_index=True)

wno_players['id'] = wno_players['id'].astype(int)
wno_players.tail()

,defence/clean_sheets,defence/goals_conceded,defence/tackles,defence/tackle_success_%,defence/last_man_tackles,defence/blocked_shots,defence/interceptions,defence/clearances,defence/headed_clearance,defence/clearances_off_line,...,teamplay/passes,teamplay/crosses,teamplay/accurate_long_balls,defence,attack,teamplay,goalkeeping,discipline,id,season
3056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445044,2020/21
3057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,441192,2020/21
3058,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-1,2019/20
3059,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-1,2020/21
3060,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-1,2021/22


In [23]:
# Match data for ordering
copy_ordered_epl = ordered_epl.copy()
for i in range(len(all_pos)):
    copy_ordered_epl = copy_ordered_epl.merge(wno_players, how='left', left_on=[all_pos[i], 'season'], 
                                              right_on=['id', 'season'], suffixes=(f"_{i-1}", f"_{i}"))
re_cols = dict(zip(copy_ordered_epl.columns[-4:], df_player.drop(columns=['season', 'name']).columns+'_26'))
copy_ordered_epl.rename(columns=re_cols, inplace=True)
copy_ordered_epl.head()

,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,home/mf_2,home/mf_3,...,defence/duels_lost,teamplay/passes,teamplay/crosses,teamplay/accurate_long_balls,defence,attack,defence/clean_sheets_26,defence/goals_conceded_26,defence/tackles_26,defence/tackle_success_%_26
0,37096.0,219924.0,81012.0,55459.0,166640.0,-1.0,204480.0,101537.0,86934.0,57531.0,...,0.0,1097.0,88.0,0.0,4.277778,15.027500,138.360000,0.0,6.50,103025
1,37915.0,55605.0,173904.0,38290.0,158534.0,-1.0,157668.0,45268.0,231372.0,62974.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1
2,116535.0,169187.0,122798.0,171287.0,97032.0,-1.0,56979.0,41733.0,116643.0,-1.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1
3,98747.0,17761.0,51927.0,39487.0,68983.0,-1.0,60586.0,60551.0,433154.0,40145.0,...,0.0,248.0,13.0,0.0,1.888889,7.594167,30.696667,0.0,6.75,200439
4,51940.0,95658.0,214590.0,184667.0,106760.0,-1.0,74208.0,195851.0,176297.0,156689.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1


In [24]:
# Check if whether missing records
count_null = 0
null_col = []
for i in range(copy_ordered_epl.shape[0]):
    if copy_ordered_epl.iloc[i].isnull().sum() > 0:
        count_null += 1
        null_col.append(i)
count_null

0

**NOTE:** Column `0_0`means performance index $0^{th}$ with position $0^{th}$.

In [25]:
# Prepare data for machine learning
dropped_copy_ordered_epl = copy_ordered_epl.copy()
dropped_copy_ordered_epl.drop(columns=dropped_copy_ordered_epl.columns[:29], inplace=True)
dropped_copy_ordered_epl.drop(columns=[col for col in dropped_copy_ordered_epl.columns if 'id' in col], inplace=True)
dropped_copy_ordered_epl.columns[:40] 

Index(['home_result', 'season', 'defence/clean_sheets_0',
       'defence/goals_conceded_0', 'defence/tackles_0',
       'defence/tackle_success_%_0', 'defence/last_man_tackles_0',
       'defence/blocked_shots_0', 'defence/interceptions_0',
       'defence/clearances_0', 'defence/headed_clearance_0',
       'defence/clearances_off_line_0', 'defence/successful_50/50s_0',
       'defence/aerial_battles_won_0', 'defence/aerial_battles_lost_0',
       'defence/own_goals_0', 'defence/errors_leading_to_goal_0',
       'teamplay/goals_0', 'teamplay/assists_0', 'teamplay/passes_per_match_0',
       'teamplay/big_chances_created_0', 'teamplay/cross_accuracy_%_0',
       'teamplay/through_balls_0', 'discipline/yellow_cards_0',
       'discipline/red_cards_0', 'discipline/fouls_0', 'attack/goals_0',
       'attack/goals_per_match_0', 'attack/headed_goals_0',
       'attack/goals_with_right_foot_0', 'attack/goals_with_left_foot_0',
       'attack/penalties_scored_0', 'attack/freekicks_scored_0',


In [46]:
# Exclude anchor columns (OPTIONAL, for original data)
for grp in skill_grps:
    dropped_copy_ordered_epl = dropped_copy_ordered_epl.drop([col for col in dropped_copy_ordered_epl if col.startswith(grp+"_")], axis=1)
[col for col in dropped_copy_ordered_epl if col.startswith(grp+"_")]

[]

In [47]:
# Check number of instances
len(dropped_copy_ordered_epl)

1076

## 3 - Save the Work

### 3.1 - Setup for Saving

In [48]:
# Where to save dataframe
def save_df(df, path, extension="csv", index=False):
    df.to_csv(path, index=index)

### 3.2 - Save the Data

In [50]:
# Define the final data
df_final_integrate = dropped_copy_ordered_epl.copy()
df_final_integrate.head()

,home_result,season,defence/clean_sheets_0,defence/goals_conceded_0,defence/tackles_0,defence/tackle_success_%_0,defence/last_man_tackles_0,defence/blocked_shots_0,defence/interceptions_0,defence/clearances_0,...,defence/duels_lost,teamplay/passes,teamplay/crosses,teamplay/accurate_long_balls,defence,attack,defence/clean_sheets_26,defence/goals_conceded_26,defence/tackles_26,defence/tackle_success_%_26
0,lose,2019/20,5.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1097.0,88.0,0.0,4.277778,15.027500,138.360000,0.0,6.50,103025
1,win,2019/20,6.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1
2,win,2019/20,13.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1
3,win,2019/20,11.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,248.0,13.0,0.0,1.888889,7.594167,30.696667,0.0,6.75,200439
4,win,2019/20,13.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1


In [51]:
# Save the final data
save_df(df_final_integrate, os.path.join(PRJ_ROOT_DIR, "data", "tabular", "integrate", "matches.csv")) # <-- change path

In [52]:
# Define matches with player id
df_match_with_player_id = copy_ordered_epl.copy()
df_match_with_player_id.head()

,home/gk_0,home/df_0,home/df_1,home/df_2,home/df_3,home/df_4,home/mf_0,home/mf_1,home/mf_2,home/mf_3,...,defence/duels_lost,teamplay/passes,teamplay/crosses,teamplay/accurate_long_balls,defence,attack,defence/clean_sheets_26,defence/goals_conceded_26,defence/tackles_26,defence/tackle_success_%_26
0,37096.0,219924.0,81012.0,55459.0,166640.0,-1.0,204480.0,101537.0,86934.0,57531.0,...,0.0,1097.0,88.0,0.0,4.277778,15.027500,138.360000,0.0,6.50,103025
1,37915.0,55605.0,173904.0,38290.0,158534.0,-1.0,157668.0,45268.0,231372.0,62974.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1
2,116535.0,169187.0,122798.0,171287.0,97032.0,-1.0,56979.0,41733.0,116643.0,-1.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1
3,98747.0,17761.0,51927.0,39487.0,68983.0,-1.0,60586.0,60551.0,433154.0,40145.0,...,0.0,248.0,13.0,0.0,1.888889,7.594167,30.696667,0.0,6.75,200439
4,51940.0,95658.0,214590.0,184667.0,106760.0,-1.0,74208.0,195851.0,176297.0,156689.0,...,-100.0,-100.0,-100.0,-100.0,-100.000000,-100.000000,-100.000000,-100.0,-100.00,-1


In [53]:
# Save the final data
save_df(df_match_with_player_id, os.path.join(PRJ_ROOT_DIR, "data", "tabular", "integrate", "matches_with_playerid.csv"))  # <-- change path